<a href="https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/04PyTorchTEXT_seq2seq_translation_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
    
このファイルは PyTorch のチュートリアルにあるファイル <https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html> を翻訳して，加筆修正したもの
です。

すぐれたチュートリアルの内容，コードを公開された Sean Robertson と PyTorch 開発陣に敬意を表します。

- Original: 
- Date: 2020-0811
- Translated and modified: Shin Asakawa <asakawa@ieee.org>

---

In [ ]:
# from https://github.com/dmlc/xgboost/issues/1715
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
%matplotlib inline

# ゼロからの NLP: 注意つきの seq2seq ネットワークによる翻訳

<!--
# NLP From Scratch: Translation with a Sequence to Sequence Network and Attention
-->

Author: [Sean Robertson](https://github.com/spro/practical-pytorch)

これは "NLP From Scratch" を行うチュートリアルの第 3 回目で，最後のチュートリアルです。
ここでは，独自のクラスと関数を書いて NLP モデル課題を実行するためにデータを前処理するようにしています。
このチュートリアルを終えた後は，このチュートリアルに続く 3 つのチュートリアルで `torchtext`がどのようにしてこの前処理の多くを処理できるのかを学んでいただければと思います。

<!--
This is the third and final tutorial on doing "NLP From Scratch", where we write our own classes and functions to preprocess the data to do our NLP modeling tasks. 
We hope after you complete this tutorial that you'll proceed to learn how `torchtext` can handle much of this preprocessing for you in the 
three tutorials immediately following this one.

In this project we will be teaching a neural network to translate from French to English.
-->

このプロジェクトでは フランス語から英語に翻訳するためのニューラルネットワークを教えます。

```
    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .
```

... to varying degrees of success.

<!--
This is made possible by the simple but powerful idea of the [sequence to sequence network](https://arxiv.org/abs/1409.3215), in which two recurrent neural networks work together to transform one sequence to another. 
An encoder network condenses an input sequence into a vector, and a decoder network unfolds that vector into a new sequence.
-->

これは 2 つのリカレントニューラルネットワーク が協力して 1 つの系列を別のシーケンスに変換する [sequence to sequence network](https://arxiv.org/abs/1409.3215) という簡潔でありながら強力なアイデアによって可能になりました。
エンコーダネットワークは入力系列をベクトルに凝縮し デコーダネットワークはそのベクトルを新しい系列に展開します。

<!--
.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:
-->

<img src="https://pytorch.org/tutorials/_images/seq2seq.png">

<!--
To improve upon this model we'll use an [attention mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder learn to focus over a specific range of the input sequence.
-->

このモデルを改良するために [注意機構](https://arxiv.org/abs/1409.0473) を使用します。
これは デコーダが入系列の特定の範囲に焦点を合わせることを学習することを可能にします。

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and understand Tensors:

- [https://pytorch.org/] PyTorch のインストール <!--For installation instructions-->
- [ディープラーニング 60 分早わかり](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)<!-- :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general-->
- [例題による PyTorch](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html) <!-- :doc:`/beginner/pytorch_with_examples` for a wide and deep overview-->
- 以前のチュートリアル <!--:doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user-->
    1. [テンソル Tensors](https://pytorch.org/tutorials/beginner/former_torchies/tensor_tutorial_old.html)
    2. [自動微分 Autograd](https://pytorch.org/tutorials/beginner/former_torchies/autograd_tutorial_old.html)
    3. [nn パッケージ nn package](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html)
    4. [マルチ GPU の例 Multi-GPU examples](https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html)

系列-to-系列ネットワークとその動作について知っておくことは有益です:
<!--
It would also be useful to know about Sequence to Sequence networks and how they work:
-->

- [統計的機械翻訳のための RNN 符号化-復号化モデルを用いた句表現の学習 Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
- [ニューラルネットワークによる系列-to-系列学習 Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
- [割当と翻訳の結合学習によるニューラル機械翻訳 Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)
- [ニューラル会話モデル A Neural Conversational Model](https://arxiv.org/abs/1506.05869)

また，前回までのチュートリアル [文字ベースの RNN による苗字分類](https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/02PyTorchTEXT_char_rnn_classification_tutorial.ipynb) と [文字ベース RNN による苗字生成](https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/03PyTorchTEXT_char_rnn_generation_tutorial.ipynb) 
も 系列-to-系列 モデルの理解に役立つでしょう。

<!--
You will also find the previous tutorials on :doc:`/intermediate/char_rnn_classification_tutorial` and :doc:`/intermediate/char_rnn_generation_tutorial` helpful as those concepts are very similar to the Encoder and Decoder models, respectively.
-->

更に，上記のような概念を提案した論文を以下に挙げます:
<!--And for more, read the papers that introduced these topics:-->

- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
- [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
- [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)
- [A Neural Conversational Model](https://arxiv.org/abs/1506.05869)

**必要条件**
<!--**Requirements**-->


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## データファイルのロード
<!--## Loading data files-->

<!--
The data for this project is a set of many thousands of English to French translation pairs.

[This question on Open Data Stack Exchange](https://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages) pointed me to the open translation site https://tatoeba.org/ which has downloads available at https://tatoeba.org/eng/downloads - and better yet, someone did the extra work of splitting language pairs into individual text files here: https://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:
-->

このプロジェクトのデータは，何千もの英語からフランス語への翻訳ペアのセットです。

[Open Data Stack Exchange でのこの質問](https://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages) で
オープン翻訳サイト [tatoeba.org](https://tatoeba.org/) を教えてもらいました。
<https://tatoeba.org/eng/downloads> からダウンロードできます。
加えて，ソース言語と目標翻訳言語の対を分割した個々のファイルは <https://www.manythings.org/anki/> から入手可能です

英語からフランス語へのペアを保存するには大きすぎるので，続ける前に ``data/eng-fra.txt`` にダウンロードしてください。
このファイルはタブで区切られた翻訳対のリストです。

```
    I am cold.    J'ai froid.
```

### 覚書:
[https://download.pytorch.org/tutorial/data.zip](https://download.pytorch.org/tutorial/data.zip) をダウンロードして，カレントディレクトリに解凍してください。

<!--Note::
Download the data from [here](https://download.pytorch.org/tutorial/data.zip)  and extract it to the current directory.
-->

<!--
Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as a one-hot vector, or giant vector of zeros except for a single one (at the index of the word). 
Compared to the dozens of characters that might exist in a language, there are many many more words, so the encoding vector is much larger.
We will however cheat a bit and trim the data to only use a few thousand words per language.
-->

文字レベル RNN チュートリアルで使用された文字符号化法と同様，言語の各単語を 1 つのホットベクトル，つまり 1 つ (単語のインデックス) を除き，全て 0 の巨大なベクトルとして表現します。
言語に存在する数十種しなかい文字に比べて，数多くの単語があるので，符号化ベクトルははるかに大きくなります。
しかし，我々は少しごまかして，言語ごとに数千語しか使わないようにデータをトリミングします。

<!--
.. figure:: /_static/img/seq-seq-images/word-encoding.png
   :alt:
-->

<img src="https://pytorch.org/tutorials/_images/word-encoding.png">




<!--
We'll need a unique index per word to use as the inputs and targets of the networks later. 
To keep track of all this we will use a helper class called ``Lang`` which has word → index (``word2index``) and index → word (``index2word``) dictionaries, as well as a count of each word ``word2count`` to use to later replace rare words.
-->

後でネットワークの入力とターゲットとして使用するためには，単語ごとに一意のインデックスが必要になります。
これを管理するために ``Lang`` というヘルパークラスを使います。
このクラスには単語→インデックス (``word2index``)  と インデックス→ワード (``index2word``) の辞書があります。


In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip > /dev/null
!unzip data.zip > /dev/null

--2020-08-11 06:16:02--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.226.50.24, 13.226.50.98, 13.226.50.63, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.50.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  13.0MB/s    in 0.2s    

2020-08-11 06:16:03 (13.0 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

<!--
The files are all in Unicode, to simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.
-->

ファイルはすべて Unicode で作成されています。
単純化するために Unicode の文字を ASCII に変え，すべてを小文字にし，ほとんどの句読点をトリミングします。


In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

<!--
To read the data file we will split the file into lines, and then split lines into pairs. 
The files are all English → Other Language, so if we want to translate from Other Language → English I added the ``reverse`` flag to reverse the pairs.
-->

データファイルを読み込むには，ファイルを行に分割し，行をペアに分割します。
ファイルは全て 英語→他言語 なので，他言語→英語 に翻訳したい場合は ペアを逆にするために ``reverse`` フラグを付けてみました。


In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

<!--
Since there are a *lot* of example sentences and we want to train something quickly, we'll trim the data set to only relatively short and simple sentences. 
Here the maximum length is 10 words (that includes ending punctuation) and we're filtering to sentences that translate to the form "I am" or "He is" etc. (accounting for apostrophes replaced earlier).
-->

たくさんの例文があり，素早く何かを訓練したいので，比較的短くて単純な文だけにデータセットをトリミングします。
ここでは，最大の長さは 10語 (語尾の句読点を含む) で， "I am" や "He is" などの形に変換される文にフィルタリングしています。
(以前に置き換えられたアポストロフィを考慮しています)



In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

<!--
The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs
-->

データを作成するための全行程は:

- テキストファイルを読み込んで行に分割し，行をペアに分割する
- テキストを正規化し，長さと内容でフィルタリングする
- 文章をペアにして単語リストを作る




In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['nous ne nous y rendons pas .', 'we re not going .']


# Seq2Seq モデル
<!--
# The Seq2Seq Model
-->

<!--
A Recurrent Neural Network, or RNN, is a network that operates on a sequence and uses its own output as input for subsequent steps.
-->

リカレントニューラルネットワーク（Recurrent Neural Network: RNN）は，系列データを操作し，モデルの出力を後続のステップの入力として使用するネットワークです。

<!--
A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215), or seq2seq network, or [Encoder Decoder network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.
-->

Sequence to Sequence network](https://arxiv.org/abs/1409.3215)，または seq2seq ネットワーク，または [符号化 復号化 ネットワーク Encoder Decoder network](https://arxiv.org/pdf/1406.1078v3.pdf) は，エンコーダとデコーダと呼ばれる 2 つの RNN からなるモデルです。
エンコーダは入力シーケンスを読み込んで 1 つのベクトルを出力し，デコーダはそのベクトルを読み込んで出力系列を生成します。

<!--
.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:
-->

<img src="https://pytorch.org/tutorials/_images/seq2seq.png">

<!--
Unlike sequence prediction with a single RNN, where every input corresponds to an output, the seq2seq model frees us from sequence length and order, which makes it ideal for translation between two languages.
-->

すべての入力が出力に対応する単一の RNN による系列予測とは異なり，seq2seq モデルは系列長や順序から解放されるので 2 つの言語間の翻訳に最適です。

<!--
Consider the sentence "Je ne suis pas le chat noir" → "I am not the black cat". 
Most of the words in the input sentence have a direct translation in the output sentence, but are in slightly different orders, e.g. "chat noir" and "black cat". Because of the "ne/pas" construction there is also one more word in the input sentence. 
It would be difficult to produce a correct translation directly from the sequence of input words.
-->

"Je ne suis pas le chat noir" →「私は黒猫ではない」という文を考えてみましょう。
入力文のほとんどの単語は出力文では直訳されています。"chat noir" と "black cat" のように少し順番が異なります。
"ne/pas" の構造のため，入力文にはもう一つ単語があります。
入力された単語の並びから直接正しい翻訳を作成するのは難しいでしょう。

<!--
With a seq2seq model the encoder creates a single vector which, in the ideal case, encodes the "meaning" of the input sequence into a single vector — a single point in some N dimensional space of sentences.
-->

seq2seq モデルでは，エンコーダは 1 つのベクトルを作成し，理想的なケースでは，入力文の「意味」を 1 つのベクトルに符号化します。


## 符号化器 エンコーダ
<!--
## The Encoder
-->

<!--
The encoder of a seq2seq network is a RNN that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.
-->

seq2seq ネットワークの符号化器 (エンコーダ) は RNN であり，
入力文の各単語に対して何らかの値を出力します。
各入力語に対してエンコーダはベクトルと隠れ層の状態を出力します。
そして，隠れ層の状態を次の入力語として使用します。

<!--
.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:
-->

<img src="https://pytorch.org/tutorials/_images/encoder-network.png">




In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## 復号化器 デコーダ
<!--
## The Decoder
-->

復号化器 デコーダは，別の RNN であり，符号化器の出力ベクトルを入力とし，出力として翻訳文となる単語系列を出力します。
<!--
The decoder is another RNN that takes the encoder output vector(s) and outputs a sequence of words to create the translation.
-->



### 簡単なデコーダ
<!--
### Simple Decoder
-->

<!--
In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes context from the entire sequence. 
This context vector is used as the initial hidden state of the decoder.
-->

最も簡単な seq2seq デコーダは，エンコーダの最後の出力のみを使用します。
この最後の出力は，系列全体の内容を符号化するので，*コンテキストベクトル* と呼ばれることもあります。
このコンテキストベクトルはデコーダの最初の隠れ層状態として使われます。

At every step of decoding, the decoder is given an input token and hidden state. 
The initial input token is the start-of-string ``<SOS>`` token, and the first hidden state is the context vector (the encoder's last hidden state).

デコードの各ステップで，デコーダには入力トークンと隠れ層の状態が与えられます。
最初の入力トークンは文字列の先頭の ``<SOS>`` トークンであり，最初の隠れ層状態はコンテキストベクトル（エンコーダの最後の隠された状態）です。

<!--
.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:
-->

<img src="https://i.imgur.com/1152PYf.png">


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<!--
I encourage you to train and observe the results of this model, but to save space we'll be going straight for the gold and introducing the Attention Mechanism.
-->

このモデルの結果を訓練して観察することをお勧めします。
ですがスペース節約のため，まっすぐ直行して注意機構の導入に直行します。




### 注意復号化器 Attention Decoder
<!--
### Attention Decoder
-->

<!--
If only the context vector is passed betweeen the encoder and decoder, that single vector carries the burden of encoding the entire sentence.
-->

符号化器と復号化器との間に文脈ベクトルだけが渡される場合，その単一のベクトルが文全体をエンコードするという負担を負うことになります。

<!--
Attention allows the decoder network to "focus" on a different part of the encoder's outputs for every step of the decoder's own outputs. 
First we calculate a set of *attention weights*. These will be multiplied by the encoder output vectors to create a weighted combination. 
The result (called ``attn_applied`` in the code) should contain information about that specific part of the input sequence, and thus help the decoder choose the right output words.
-->

注意により，復号化ネットワークは復号化器自身の出力のステップごとに，符号化器の出力の異なる部分に「焦点を当てる」ことができます。
最初に *注意の重み attention weight* のセットを計算します。
これらを符号化器の出力ベクトルと掛け合わせて，重み付けされた組み合わせを作ります。
その結果 (コード中では ``attn_applied`` と呼ばれています) は，入力系列の特定の部分に関する情報を含み，復号化器が正しい出力語を選択するのに役立ちます。

<!--
.. figure:: https://i.imgur.com/1152PYf.png
   :alt:
-->
<img src="https://i.imgur.com/1152PYf.png">

<!--
Calculating the attention weights is done with another feed-forward layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to actually create and train this layer we have to choose a maximum sentence length (input length, for encoder outputs) that it can apply to. 
Sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few.
-->

注意重みの計算は，別のフィードフォワード層 ``attn`` で行われます。復号化器 デコーダの入力と隠れ層の状態を入力として使用します。
学習データにはあらゆるサイズの文が含まれているので，実際にこの層を作成して訓練するためには，この層が適用できる最大の文の長さ（エンコーダの出力のための入力の長さ）を選択しなければなりません。
最大長の文はすべての注目重みを使用し，短い文は最初の数文のみを使用します。

<!--
.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:
-->
<img src="https://pytorch.org/tutorials/_images/attention-decoder-network.png">




In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### 覚書
<!--
#### Note
-->

<!--
There are other forms of attention that work around the length limitation by using a relative position approach. 
Read about "local attention" in [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)
-->

相対位置のアプローチを使用することで長さの制限を回避する注意の他の形態があります。
「局所的注意」について [Effective Approaches to Attention-based Neural Machine Translation] (https://arxiv.org/abs/1508.04025) を参照してください。

## 訓練
<!--
## Training
-->

### 訓練データの準備
<!--
### Preparing Training Data
-->

To train, for each pair we will need an input tensor (indexes of the words in the input sentence) and target tensor (indexes of the words in the target sentence). 
While creating these vectors we will append the EOS token to both sequences.

学習には，各翻訳対について，入力テンソル (入力文に含まれる単語のインデックス) と目標テンソル (ターゲット文に含まれる単語のインデックス) が必要です。
これらのベクトルを作成しながら EOS トークンを両方のシーケンスに追加します。


In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## モデルの訓練
<!--
## Training the Model
-->

<!--
To train we run the input sentence through the encoder, and keep track of every output and the latest hidden state. Then the decoder is given the ``<SOS>`` token as its first input, and the last hidden state of the encoder as its first hidden state.
-->

訓練するには，入力文をエンコーダを通して実行し，すべての出力と最新の隠さ層の状態を保持しておきます。
そして，デコーダには最初の入力として ``<SOS>`` トークンが与えられ，エンコーダの最後の隠れ層の状態が最初の隠れ層状態として与えられます。

<!--
"Teacher forcing" is the concept of using the real target outputs as each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but [when the trained network is exploited, it may exhibit instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf). 
-->

「教師強制」 とは，デコーダの推測を次の入力として使うのではなく，実際の目標とする出力を次の入力として使うという概念です。
教師強制を使うと収束が速くなります。ですが，[訓練されたネットワークが悪用されると不安定性を示す可能性があります](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf)。

<!--
You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick up" the meaning once the teacher tells it the first few words, but it has not properly learned how to create the sentence from the translation in the first place.
-->

教師強制したネットワークの出力を観察することができます。
直感的には，出力された文法を表現することを学んでいて，
教師が最初の数語を教えてくれれば意味を「拾う」ことができますが，そもそも翻訳から文を作る方法を正しく学んでいません。

<!--
Because of the freedom PyTorch's autograd gives us, we can randomly choose to use teacher forcing or not with a simple if statement. 
Turn ``teacher_forcing_ratio`` up to use more of it.
-->

PyTorch の 自動微分 が与えてくれる自由度のおかげで，簡単な if 文で教師強制を使うか使わないかをランダムに選択することができます。
教師強制を更に使うためには ``teacher_forcing_ratio`` を上げてください。



In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

<!--
This is a helper function to print time elapsed and estimated time remaining given the current time and progress %.
-->
以下は，現在の時刻と進行度％を与えられた経過時間と推定残り時間を表示するヘルパー関数です。


In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

<!--
The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (% of examples, time so far, estimated time) and average loss.
-->

トレーニング全体の流れは以下のとおりです:

- タイマーを開始
- オプティマイザと収束基準を初期化
- 訓練データ対のセットを作成
- プロット用の空の損失配列を開始

そして 何度も ``train`` を呼び出し、時々進捗状況を表示します (例文を処理した割合(％), これまでの経過時間，推定終了時間）と平均損失を表示します。


In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

## 結果のプロット
<!--
## Plotting results
-->

<!--Plotting is done with matplotlib, using the array of loss values ``plot_losses`` saved while training.-->
学習中に保存された損失値の配列 ``plot_losses`` を用いて matplotlib を用いてプロットを行います．


In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# 評価
<!--
# Evaluation
-->

<!--
Evaluation is mostly the same as training, but there are no targets so we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it predicts the EOS token we stop there. We also store the decoder's attention outputs for display later.
-->

評価は訓練とほとんど同じです。ですがターゲットがないので，各ステップごとにデコーダの予測値を自分自身にフィードバックします。
デコーダが単語を予測するたび，その予測値を出力文字列に追加し，EOS トークンを予測した場合はそこで停止します。
また，後で表示するためにデコーダ注意の出力を保存します。


In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

<!--
We can evaluate random sentences from the training set and print out the input, target, and output to make some subjective quality judgements:
-->

訓練セットからランダムな文章を評価し，入力，ターゲット，出力を印字することで，いくつかの主観的な質的質判断を行うことができます:




In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## 訓練と評価
<!--
## Training and Evaluating
-->

<!--
With all these helper functions in place (it looks like extra work, but it makes it easier to run multiple experiments) we can actually initialize a network and start training.
-->

これらのヘルパー関数がすべて揃っていれば (余計な作業のように見えますが，複数の実験を行うのが簡単になります)，実際にネットワークを初期化して学習を開始することができます。

<!--
Remember that the input sentences were heavily filtered. For this small dataset we can use relatively small networks of 256 hidden nodes and a single GRU layer. 
After about 40 minutes on a MacBook CPU we'll get some reasonable results.
-->

入力文は大きくフィルタリングされていることを覚えておいてください。
この小さなデータセットでは 256 個の隠れノードと 1 つの GRU層 からなる比較的小さなネットワークを使うことができます。
MacBook の CPU で 約40分後には それなりの結果が得られるでしょう。

#### 覚書:
<!---.. Note::-->
<!--
If you run this notebook you can train, interrupt the kernel, evaluate, and continue training later. Comment out the lines where the encoder and decoder are initialized and run ``trainIters`` again.
-->

このノートブックを実行すると，訓練をしたり，カーネルを中断したり，評価したり，後で訓練を続けたりすることができます。
エンコーダとデコーダが初期化されている行をコメントアウトして，再度  ``trainIters`` を実行してください。




In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

2m 10s (- 30m 31s) (5000 6%) 2.8471
4m 17s (- 27m 52s) (10000 13%) 2.2871
6m 24s (- 25m 36s) (15000 20%) 1.9710
8m 30s (- 23m 23s) (20000 26%) 1.7186
10m 35s (- 21m 10s) (25000 33%) 1.5460
12m 39s (- 18m 58s) (30000 40%) 1.3600


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

## 注意の視覚化
<!--
## Visualizing Attention
-->

<!--
A useful property of the attention mechanism is its highly interpretable outputs. 
Because it is used to weight specific encoder outputs of the input sequence, we can imagine looking where the network is focused most at each time step.
-->

注意メカニズムの有用な特性は、その高度に解釈可能な出力です。
これは入力系列の特定のエンコーダ出力を重み付けするために使われているので，各タイムステップでネットワークがどこに最も集中しているかを見ることができます。

<!--
You could simply run ``plt.matshow(attentions)`` to see attention output displayed as a matrix, with the columns being input steps and rows being output steps:
-->

単に ``plt.matshow(attentions)`` を実行すると，注意の出力が行列として表示されます。
列が入力ステップ，行が出力ステップとなります。


In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

<!--
For a better viewing experience we will do the extra work of adding axes and labels:
-->
より見やすくするために 軸とラベルの追加作業を行います:


In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

## 演習
<!--
## Exercises
-->

<!--
- Try with a different dataset
    - Another language pair
    - Human → Machine (e.g. IOT commands)
    - Chat → Response
    - Question → Answer
-->

- 別のデータセットで試してみよ
    - 別の言語ペア
    - 人間 → 機械（IoTコマンドなど)
    - チャット → レスポンス
    - 質問 → 回答

<!--
-  Replace the embeddings with pre-trained word embeddings such as word2vec or GloVe
-  Try with more layers, more hidden units, and more sentences. Compare the training time and results.
-  If you use a translation file where pairs have two of the same phrase  (``I am test \t I am test``), you can use this as an autoencoder. 
-->

- 埋め込みを word2vec や GloVe のような学習済みの単語埋め込みに置き換えよ。
- より多くの層，より多くの隠れユニット，より多くの文で試してみよ。学習時間と結果を比較せよ
- 同じフレーズのペアが 2 つある翻訳ファイル (``I am test \t I am test``) を使っている場合は これをオートエンコーダーとして使うことができる。

<!--
Try this:
    - Train as an autoencoder
    - Save only the Encoder network
    - Train a new Decoder for translation from there
-->

以下を試してみよ
    - オートエンコーダーとして訓練する
    - エンコーダネットワークのみ保存する
    - そこから翻訳のための新しいデコーダーを訓練する

